In [ ]:
import os
# Victim models from huggingface model hub
model_paths = ['textattack/bert-base-uncased-ag-news',
            'textattack/bert-base-uncased-MNLI',
            'textattack/bert-base-uncased-SST-2',

            'textattack/albert-base-v2-ag-news',
            'textattack/albert-base-v2-SST-2',
            'textattack/roberta-base-MNLI',
            "howey/bert-base-uncased-mnli",
            'textattack/roberta-base-ag-news',
            'textattack/roberta-base-SST-2',
            'Alireza1044/albert-base-v2-mnli']

In [ ]:
os.system(f"nohup python RunTPGD.py \
                --task sst2\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-SST-2 \
                    \
                --victim_model_checkpoint textattack/roberta-base-SST-2\
                --victim_tokenizer_checkpoint textattack/roberta-base-SST-2\
                    \
                --data_local_path ~/datasets/sst2\
                --start 0  --end 912\
                --cuda_device 0 --victim_device -1\
                --perturb_layer 0 --decode_layer 10\
                --num_seg_steps 100 --num_adv_steps 20\
                --adv_lr5 --init_mag 3 --decode_weight -0.5\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                --eval_lower_limit 0.60\
                >SST2Exp/BertOnRoberta-SST2-Layer-Adv-Dw-USE-FULL.out 2>&1 &")

## Parameters Description ##

# task: Task to run PandR on

# tokenizer_checkpoint: Tokenizer used to build encoded dataset

# decode_mode: Specify the behavior mode of prober, should be correspond to the type of victim model

# model_checkpoint: Model name used in initializing model with AutoModel.from_pretrained()

# victim_model_checkpoint victim model name used in initializing model with AutoModel.from_pretrained()

# vvictim_tokenizer_checkpoint: Tokenizer used to tokenize adv samples to feed in victim model

# data_local_path: Local path to the dataset saved with Dataset.save_to_disk()

# start / end: Run attack on dataset[start:end]

# cuda_device: Specify CUDA device. CPU mode is not supported yet.

# victim_device: Default value -1 means initializing victim model on the same CUDA device as inferrence model.
# Set to a valid value will move the vectim model to sepecific CUDA device
# Designed for GPU memory insufficient cases

# perturb_layer: The index of layer to add perturbation to. 0 performs best in most cases.

# decode_layer: The index of layer to do hidden presentation reconstruction on.

# num_seg_steps: Maximum number of iterations allowed for each input sample. 
# Corresponds to MaxIter in the paper.

# num_adv_steps: Maximum steps of perturbation accumulation allowed in one iteration.
# Corresponds to MaxStep in the paper.

# adv_lr: The 2-norm value of a single perturbation, size of an adversarial perturbation step.

# init_mag: The 2-norm value of the random initialized perturbation.

# decode_weight: The value of β described in the paper. Loss = task_Loss + β * decode_Loss
# Designed to keep the adversarial text from straying too far from original text.

# bs_lower_limit: The lowest USE score between original & adversarial text. 
# Sample that gets lower USE score than this value will be ignored.

# bs_upper_limit: The highest USE score between original & adversarial text. 
# Sample that gets higher USE score than this value will trigger an early-stop of the attack process.

# seed: Default value: 114514. Random seed used in attack process.

## Extra Parameters Description ##

# local_victim: Path to loacally saved victim model. Model should have been saved with torch.save(model, path)

# local_model: Path to loacally saved inferrence model. Model should have been saved with torch.save(model, path)

# stop_random_cover: Default value: False. Set this to True to stop replacing random token with [MASK] in the beginning of each iteration.

In [2]:
#BERT on Roberta SST2
device = 1
for layer in [10]:
    for adv_lr in [5]:
        for dw in [0.5]:
            os.system(f"nohup python RunTPGD.py \
                --task sst2\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-SST-2 \
                    \
                --victim_model_checkpoint textattack/roberta-base-SST-2\
                --victim_tokenizer_checkpoint textattack/roberta-base-SST-2\
                    \
                --data_local_path ~/datasets/sst2\
                --start 0  --end 912\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 100 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                --eval_lower_limit 0.60\
                >SST2Exp/BertOnRoberta-SST2-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-FULL.out 2>&1 &")
            import time
            device += 1

In [2]:
#BERT on Albert SST2
device = 1
for layer in [10]:
    for adv_lr in [5]:
        for dw in [0.5]:
            os.system(f"nohup python RunTPGD.py \
                --task sst2\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-SST-2 \
                    \
                --victim_model_checkpoint textattack/albert-base-v2-SST-2\
                --victim_tokenizer_checkpoint textattack/albert-base-v2-SST-2\
                    \
                --data_local_path ~/datasets/sst2\
                --start 0  --end 100\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 100 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                --eval_lower_limit 0.75\
                >SST2Exp/100-BertOnAlbert-SST2-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-FULL.out 2>&1 &")
            import time
            device += 1

In [1]:
#BERT on Bert SST2
device = 0
for layer in [10]:
    for adv_lr in [5]:
        for dw in [0.5]:
            os.system(f"nohup python RunTPGD.py \
                --task sst2\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-SST-2 \
                    \
                --victim_model_checkpoint textattack/bert-base-uncased-SST-2\
                --victim_tokenizer_checkpoint textattack/bert-base-uncased-SST-2\
                    \
                --data_local_path ~/datasets/sst2\
                --start 0  --end 912\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 100 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --eval_lower_limit 0.70\
                --target_metric use\
                --local_model /home/phantivia/models/bert-base-uncased-SST-2.model\
                >SST2Exp/BertOnBert-SST2-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-FULL.out 2>&1 &")

            import time
            device += 1

In [6]:
#BERT on Roberta AG
device = 0
import time
for layer in [8]:
    device = 0
    for adv_lr in [5, 10, 15]:
        for dw in [0.5, 1.0]:
            os.system(f"nohup python RunTPGD.py \
                --task ag_news\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-ag-news \
                    \
                --victim_model_checkpoint textattack/roberta-base-ag-news\
                --victim_tokenizer_checkpoint textattack/roberta-base-ag-news\
                    \
                --data_local_path ~/datasets/ag_news\
                --start 0  --end 100\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 100 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                >logs/BertOnRoberta-AG-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-100.out 2>&1 &")
            
            device += 1

In [9]:
#BERT on Albert AG
device = 0
import time
for layer in [8, 9]:
    for adv_lr in [10]:
        for dw in [0.5, 1.0]:
            os.system(f"nohup python RunTPGD.py \
                --task ag_news\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-ag-news \
                    \
                --victim_model_checkpoint textattack/albert-base-v2-ag-news\
                --victim_tokenizer_checkpoint textattack/albert-base-v2-ag-news\
                    \
                --data_local_path ~/datasets/ag_news\
                --start 0  --end 1000\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 100 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                >logs/BertOnAlbert-AG-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-100.out 2>&1 &")
            
            device += 1

In [2]:
#BERT on BERT AG
device = 1
import time
for layer in [7]:
    for adv_lr in [10]:
        for dw in [0.5]:
            os.system(f"nohup python RunTPGD.py \
                --task ag_news\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-ag-news \
                    \
                --victim_model_checkpoint textattack/bert-base-uncased-ag-news\
                --victim_tokenizer_checkpoint textattack/bert-base-uncased-ag-news\
                    \
                --data_local_path ~/datasets/ag_news\
                --start 0  --end 1000\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 150 --num_adv_steps 15\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.85\
                --target_metric use\
                --eval_lower_limit 0.50\
                --local_model ~/models/bert-base-uncased-ag-news.model\
                >SST2Exp/BertOnBert-AG-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-FULL.out 2>&1 &")
            
            device += 1

In [2]:
#BERT on Albert MNLI
device = 1
import time
for layer in [11]:
    for adv_lr in [1]:
        for dw in [10]:
            os.system(f"nohup python RunTPGD.py \
                --task mnli_hypothesis\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-MNLI \
                    \
                --victim_model_checkpoint Alireza1044/albert-base-v2-mnli\
                --victim_tokenizer_checkpoint albert-base-v2\
                    \
                --data_local_path ~/datasets/mnli\
                --start 0  --end 1000\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 150 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                --eval_lower_limit 0.7\
                >SST2Exp/BertOnAlbert-MNLI-Layer{layer}-Adv{adv_lr}-Dw{dw}-BS-FULL.out 2>&1 &")
            
            device += 1

In [2]:
#BERT on Bert MNLI
device = 0
import time
for layer in [12]:
    for adv_lr in [1]:
        for dw in [10]:
            os.system(f"nohup python RunTPGD.py \
                --task mnli_hypothesis\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  ishan/bert-base-uncased-mnli\
                    \
                --victim_model_checkpoint textattack/bert-base-uncased-MNLI\
                --victim_tokenizer_checkpoint bert-base-uncased\
                    \
                --data_local_path ~/datasets/mnli\
                --start 0  --end 1000\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 150 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                --eval_lower_limit 0.6\
                >SST2Exp/BertOnBert-MNLI-Layer{layer}-Adv{adv_lr}-Dw{dw}-USE-FULL.out 2>&1 &")
            
            device += 1

In [4]:
#BERT on ROBERTA MNLI
device = 1
import time
for layer in [11]:
    for adv_lr in [1]:
        for dw in [10]:
            os.system(f"nohup python RunTPGD.py \
                --task mnli_hypothesis\
                --tokenizer_checkpoint bert-base-uncased\
                --decode_mode Bert \
                --model_checkpoint  textattack/bert-base-uncased-MNLI \
                    \
                --victim_model_checkpoint textattack/roberta-base-MNLI\
                --victim_tokenizer_checkpoint roberta-base\
                    \
                --data_local_path ~/datasets/mnli\
                --start 0  --end 1000\
                --cuda_device {device} --victim_device -1\
                --perturb_layer 0 --decode_layer {layer}\
                --num_seg_steps 200 --num_adv_steps 20\
                --adv_lr {adv_lr} --init_mag 3 --decode_weight -{dw}\
                --bs_lower_limit 0.30 --bs_upper_limit 0.95\
                --target_metric use\
                --eval_lower_limit 0.4\
                >SST2Exp/BertOnRoberta-MNLI-Layer{layer}-Adv{adv_lr}-Dw{dw}-BS-FULL.out 2>&1 &")